In [83]:
%load_ext autoreload
%autoreload 2
dataDir = "E:/Work/NonoGram/"
import json
import keras
import keras.utils
import tensorflow as tf
import numpy as np
import createDigitImage
import os
import cv2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
evaluate_output_dir = "model_evaluation_result"
def evaluate_tests(model, test_data, test_labels):
    empty_test_dir()
    predictions = model.predict(test_data)
    for i in range(0, len(predictions)):
        predicted_digit = np.argmax(predictions[i])
        if (predicted_digit != test_labels[i]):
            image = np.reshape(test_data[i], (28, 28, 1))*255
            cv2.imwrite(os.path.join(evaluate_output_dir, f"{predicted_digit}_{test_labels[i]}_{i}.png"), image)

def empty_test_dir():
    if os.path.exists(evaluate_output_dir):
        for filename in os.listdir(evaluate_output_dir):
            file_path = os.path.join(evaluate_output_dir, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
    os.makedirs(evaluate_output_dir, exist_ok=True)                


In [85]:
class ErrorCallback(tf.keras.callbacks.Callback):
    def __init__(self, output_dir='error_logs'):
        print("start")
        self.failed_epochs = []  # List to store failed cases
        self.output_dir = output_dir
        self.file_path = os.path.join(self.output_dir, 'failed_epochs.json')

        # Clean the directory if it exists (remove old logs)
        if os.path.exists(self.output_dir):
            for filename in os.listdir(self.output_dir):
                file_path = os.path.join(self.output_dir, filename)
                if os.path.isfile(file_path):
                    os.remove(file_path)

        # Ensure the subdirectory exists
        os.makedirs(self.output_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        # Check for specific conditions (e.g., loss or accuracy being None)
        if logs is None or logs.get('loss') is None or logs.get('accuracy') is None:
            failed_case = {
                'epoch': epoch,
                'loss': logs.get('loss'),
                'accuracy': logs.get('accuracy')
            }
            self.failed_epochs.append(failed_case)
            print(f"Epoch {epoch} failed. Loss: {logs.get('loss')}, Accuracy: {logs.get('accuracy')}")

    def on_train_end(self, logs=None):
        print("end")
        with open(self.file_path, 'w') as f:
            json.dump(self.failed_epochs, f, indent=4)
        with open(os.path.join(self.output_dir,'hoi.txt'), 'w') as g:
            g.write("aha")

    def get_failed_epochs(self):
        return self.failed_epochs


In [86]:


def combine_data(data_set_1, data_set_2):
    (train_data, train_labels, test_data, test_labels) = data_set_1
    (extra_train_data, extra_train_labels, extra_test_data, extra_test_labels) = data_set_2

    train_data = np.concatenate([train_data, extra_train_data], axis=0)
    train_labels = np.concatenate([train_labels, extra_train_labels], axis=0)
    test_data = np.concatenate([test_data, extra_test_data], axis=0)
    test_labels = np.concatenate([test_labels, extra_test_labels], axis=0)
    return (train_data, train_labels, test_data, test_labels)

def load_data_from_mnist():
    (train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()
    train_data = train_data.astype('float32') / 255
    test_data = test_data.astype('float32') / 255
    train_data = np.expand_dims(train_data, axis=-1)
    test_data = np.expand_dims(test_data, axis=-1)
    return (train_data, train_labels, test_data, test_labels)

def create_data_from_fonts(train_count, test_count):
    extra_train_data = [createDigitImage.create_random_digit_image() for i in range(0, train_count)]
    extra_test_data = [createDigitImage.create_random_digit_image() for i in range(0, test_count)]

    train_data = np.reshape([data for (data, label) in extra_train_data], (train_count, 28, 28, 1))
    train_labels = [label for (data, label) in extra_train_data]
    test_data = np.reshape([data for (data, label) in extra_test_data], (test_count, 28, 28, 1))
    test_labels = [label for (data, label) in extra_test_data]
    return (train_data, train_labels, test_data, test_labels)


# Load the MNIST dataset from OpenCV (this loads pre-trained data)
# This is a convenient way to quickly get started with digit recognition.
def createDigitRecognizer():
    #(train_data, train_labels, test_data, test_labels) = load_data_from_mnist()
    #(train_data, train_labels, test_data, test_labels) = create_data_from_fonts(train_count = 40000, test_count = 2000)
    #(train_data, train_labels, test_data, test_labels) = create_data_from_fonts(train_count = 2000, test_count = 1000)
    (train_data, train_labels, test_data, test_labels) = combine_data(load_data_from_mnist(), create_data_from_fonts(40000, 5000))

    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        keras.layers.MaxPooling2D((2, 2)),
        #keras.layers.BatchNormalization(),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        #keras.layers.BatchNormalization(),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(256, activation='relu'),
        #keras.layers.BatchNormalization(),
        #keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])
    
    # Compile the model
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    # Train the model
    model.fit(train_data, tf.keras.utils.to_categorical(train_labels), epochs=10, batch_size=64, validation_data=(test_data, tf.keras.utils.to_categorical(test_labels)))
    model.evaluate(test_data,  tf.keras.utils.to_categorical(test_labels), verbose=2)
    evaluate_tests(model, test_data, test_labels)

    return model

import os

#import keras.saving
digitRecognizerModelFileName = f"{dataDir}/digitRecognizerMnist.keras"
digitRecognizer = createDigitRecognizer()
digitRecognizer.save(f"{dataDir}/digitRecognizerMnist.keras")

Train on 100000 samples, validate on 15000 samples
Epoch 1/10
100000/100000 [==============================] - 6s 59us/step - loss: 0.3979 - accuracy: 0.8831 - val_loss: 0.0954 - val_accuracy: 0.9702
Epoch 2/10
100000/100000 [==============================] - 6s 56us/step - loss: 0.1385 - accuracy: 0.9626 - val_loss: 0.0583 - val_accuracy: 0.9826
Epoch 3/10
100000/100000 [==============================] - 6s 56us/step - loss: 0.0933 - accuracy: 0.9745 - val_loss: 0.0364 - val_accuracy: 0.9885
Epoch 4/10
100000/100000 [==============================] - 6s 57us/step - loss: 0.0528 - accuracy: 0.9839 - val_loss: 0.0295 - val_accuracy: 0.9907
Epoch 5/10
100000/100000 [==============================] - 7s 66us/step - loss: 0.0827 - accuracy: 0.9812 - val_loss: 0.0264 - val_accuracy: 0.9907
Epoch 6/10
100000/100000 [==============================] - 6s 63us/step - loss: 0.0673 - accuracy: 0.9842 - val_loss: 0.0294 - val_accuracy: 0.9899
Epoch 7/10
100000/100000 [=============================